ДЗ по итогам лекции 8, задание 1.

Подготовить данные для модели:
    
   Анализ данных, необходимые корректировки. 
1. Обработать пропуски. 
2. Оценить выбросы. 
3. Корреляция. 
4. Тест на нормальность распределения.
5. Масштабировать данные.

Анализ задания: следует загрузить файл данных, представленный в формате csv, в датафрейм модуля pandas.
*) Далее следует провести поиск пропусков и удалить строки с пропусками.
*) Кроме того, предварительный анализ файла данных показывает, что колонка living region содержит много дублей типа "московская обл" и "обл московская" или "чеченская республика" и "чеченская респ". Придется поработать с данными этой колонки, чтобы привести систему обозначений регионов к однозначной. Поскольку регионов слишком много, что затруднит анализ корреляции данных, то мы приведем всю систему к федеральным округам и двум столицам - Москва и Санкт-Петербург.
*) Далее для оценки выбросов числовых данных следует провести оценку матожидания и дисперсии каждого столбца данных, представленного в численном виде. Все значения, отстоящие более чем на 3 выборочных стандартных отклонения, удаляем из массива данных (точнее, удаляем из выборки все объекты, для которых хотя бы в одном столбце имеются выбросы).
*) С помощью функций подключенной библиотек оценим нормальность распределений данных
*) В конце числовые колонки данных масштабируем, поделив на максимальный элемент каждой колонки

Загружаем данные.
Выводим для ознакомления заголовочную строку и первые пять строк. 

In [10]:
import pandas as pd
import sklearn as sk

data_table = pd.read_csv('credit_train.csv', sep=';', decimal = ',', encoding = 'windows-1251')
print(f'Rows in the initial table: {len(data_table)}') 
print(data_table.head())

Rows in the initial table: 170746
   client_id gender   age marital_status job_position  credit_sum  \
0          1      M   NaN            NaN          UMN    59998.00   
1          2      F   NaN            MAR          UMN    10889.00   
2          3      M  32.0            MAR          SPC    10728.00   
3          4      F  27.0            NaN          SPC    12009.09   
4          5      M  45.0            NaN          SPC         NaN   

   credit_month tariff_id  score_shk education        living_region  \
0            10       1.6        NaN       GRD   КРАСНОДАРСКИЙ КРАЙ   
1             6       1.1        NaN       NaN               МОСКВА   
2            12       1.1        NaN       NaN      ОБЛ САРАТОВСКАЯ   
3            12       1.1        NaN       NaN    ОБЛ ВОЛГОГРАДСКАЯ   
4            10       1.1   0.421385       SCH  ЧЕЛЯБИНСКАЯ ОБЛАСТЬ   

   monthly_income  credit_count  overdue_credit_count  open_account_flg  
0         30000.0           1.0                   

Видим сразу много NaN. 
Это процедура загрузки заменила пропуски.
Такие строки подлежат устранению.
Удаляем строки c NaN (то есть с отсутствовавшими изначально значениями, которые при загрузке с помощью функции read_csv прерватились в NaN) c помощью dropna:

In [11]:
data_table.dropna(inplace=True) # inplace means we change data_table itself, not create new modified one
print(data_table.head())

    client_id gender   age marital_status job_position  credit_sum  \
7           8      F  26.0            UNM          SPC     47878.0   
9          10      F  32.0            UNM          SPC     26268.0   
10         11      M  26.0            MAR          SPC     16793.0   
12         13      M  37.0            MAR          SPC     42289.0   
13         14      M  43.0            MAR          INP     57567.0   

    credit_month tariff_id  score_shk education        living_region  \
7             10       1.1   0.512525       GRD       МОСКОВСКАЯ ОБЛ   
9             10       1.1   0.465026       GRD  КРАЙ СТАВРОПОЛЬСКИЙ   
10            14       1.0   0.445430       SCH      САНКТ-ПЕТЕРБУРГ   
12            10       1.6   0.691609       SCH    ОБЛ АРХАНГЕЛЬСКАЯ   
13            10       1.1   0.341164       GRD  ХАНТЫ-МАНСИЙСКИЙ АО   

    monthly_income  credit_count  overdue_credit_count  open_account_flg  
7          60000.0           3.0                   0.0                 

Теперь следует обработать колонку living region. 
Поскольку регионов слишком много, работа сразу со всеми даст громоздкие результаты, анализ которых затруднен.
Даже Путину это было ясно еще в 2000 г., отчего вся территория России была им поделена на федеральные округа. 
На данный момент их восемь. Еще имеет смысл отдельно рассматривать Москву и Петербург, поскольку в социологическом плане они сильно отличаются от остальной России.
Поэтому мы объединим все регионы по федеральным округам плюс отдельно укажем Москву и Петербург. 
Получим ровно десять living regions:
1. Центральный ФО
2. Северо-Западный ФО
3. Южный ФО
4. Северо-Кавказский ФО
5. Приволжский  ФО
6. Уральский ФО
7. Сибирский ФО
8. Дальневосточный ФО
9. Москва
10. Санкт-Петербург

С таким количеством регионов уже можно работать.
Попутно отметим, что в таблице два раза встречается регион Россия. Придется его целенаправленно уничтожить.

In [12]:
# looking for the region 'Россия' and removing it:
indices = data_table.index[data_table['living_region'] == 'РОССИЯ']
for ind in indices:
    data_table = data_table.drop(index = ind)

# function returning the string replace_to if a data_cell contains a string from string_set
def replace_cell_value(data_cell, string_set, replace_to):
    for str in string_set:
        if str in data_cell:
            return replace_to
            break
    return data_cell # if no string in string_set is present in data_cell

# the regions of every federal district, keywords:
central_fo = ['БЕЛГОРОД', 'БРЯНСК', 'ВЛАДИМИР', 'ВОРОНЕЖ', 'ГУСЬ', 'ИВАНОВ', 'КАЛУГ', 'КАЛУЖ', 'КОСТРОМ', 'КУРСК', 'ЛИПЕЦК', 'МОСКОВСК', 'МЫТИЩ', 'ОРЕЛ', 'ОРЁЛ', 'ОРЛОВСК', 'РЯЗАН', 'СМОЛЕНСК', 'ТАМБОВ', 'ТВЕР', 'ТУЛА', 'ТУЛЬСК', 'ЯРОСЛАВ'] 
north_west_fo = ['КАРЕЛ','КОМИ','АРХАНГ','ВОЛОГ','КАЛИНИНГР','ЛЕНИНГР','МУРМАН','НОВГОРОД','ПСКОВ','НЕНЕЦК']
south_fo = ['АДЫГ','КАЛМЫК','КРЫМ','КРАСНОДАР','АСТРАХАН','ВОЛГОГРАД','РОСТОВ','СЕВАСТ']
caucas_fo =['ДАГЕСТ','ИНГУШ','КАБАРДИН','БАЛКАР','КАРАЧАЕВ','ЧЕРКЕС','ОСЕТИ','АЛАНИ','ЧЕЧЕН','ЧЕЧНЯ','СТАВРОПОЛ']
volga_fo = ['ПРИВОЛЖСК','БАШКИР','БАШКОР', 'ГОРЬКИЙ', 'ГОРЬКОВСК', 'МАРИЙ','МОРДОВ','ТАТАР','УДМУРТ','ЧУВАШ','ПЕРМ','КИРОВ','НИЖЕГОРОД','НИЖНИЙ','ОРЕНБУРГ','ПЕНЗ','САМАР','САРАТОВ','УЛЬЯН','СИМБИРС']
urals_fo =['ЕКАТЕРИНБ','СВЕРДЛОВ','КУРГАН','ТЮМЕН','ЧЕЛЯБ','ТАГИЛ','ХАНТЫ','МАНСИ','ЯМАЛ','НЕНЕЦ', '74']
siberia_fo = ['АЛТАЙ','ТЫВА','ТУВА','ТУВИН','ХАКАС','КРАСНОЯРСК','ИРКУТСК','КЕМЕРОВ','КУЗБАСС','НОВОСИБ','ОМСК', 'ЭВЕНК']
far_east_fo = ['ВЛАДИВОСТ','ХАБАРОВ','БУРЯТ','САХА','ЯКУТ','БАЙКАЛ','КАМЧАТ','ПРИМОРСК','АМУР','МАГАДАН','САХАЛИН','ЕВРЕЙ','ЧИТА', 'ЧИТИН', 'ЧУКОТ']
moskau = ['МОСКВ']
piter = ['ПЕТЕРБ', '98']
regions = [central_fo, north_west_fo, south_fo, caucas_fo, volga_fo, urals_fo, siberia_fo, far_east_fo, moskau, piter]

# regions to be put into dataset:
federal_districts = ['Центральный ФО', 'Северо-Западный ФО', 'Южный ФО', 'Северо-Кавказский ФО', 'Приволжский  ФО', 'Уральский ФО', 'Сибирский ФО', 'Дальневосточный ФО', 'Москва', 'Санкт-Петербург']

# Replacing the cells in living_region with federal districts
i = 0
for region in regions:
    data_table['living_region'] = data_table['living_region'].map(lambda x: replace_cell_value(x, region, federal_districts[i]))
    i += 1

# saving result for debugging purposes
data_table.to_csv('tmp.csv', index = False, sep=';', decimal = ',', encoding = 'windows-1251') 



Отсеиваем строки с выбросами (где значения отстоят больше чем на 3 стандартных отклонения от среднего).
Перед этим кодируем все нечисловые колонки энкодером OneHotEncoder

In [18]:
column_names = ['gender', 'marital_status', 'job_position', 'education']
for column in column_names:
    data_table[column]

encoder = OneHotEncoder() # init encoder

# Fit and transform the data
data_table['gender'] = encoder.fit_transform(data_table[['gender']])



# mean = np.mean(data)
# std_dev = np.std(data)
# z_scores = (data - mean) / std_dev
# filtered_data = data[np.abs(z_scores) < 3]
# print(f'Rows in the filtered table: {len(data_table)}') 

TypeError: sparse array length is ambiguous; use getnnz() or shape[0]

In [ ]:
len(data_table)

161329

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Example DataFrame
df = pd.DataFrame({
    'Color': ['Red', 'Blue', 'Green', 'Blue', 'Red']
})

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the data
encoded_data = encoder.fit_transform(df[['Color']])

# Get the feature names for the encoded columns
encoded_feature_names = encoder.get_feature_names_out(['Color'])

# Create a DataFrame with the encoded data
encoded_df = pd.DataFrame(encoded_data, columns = encoded_feature_names)

# Display the encoded DataFrame
print(encoded_df)

ValueError: Shape of passed values is (5, 1), indices imply (5, 3)